In [1]:
import os
import torch
import numpy as np
import setup.dataset as dataset
import setup.ResUNet as ResUNet
import setup.classifier as classifier
from torch.utils.data import SubsetRandomSampler
from PIL import Image
import matplotlib.pyplot as plt
import cv2

np.random.seed(0)
torch.manual_seed(0)

DATASET_PATH = 'dataset'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def sampler_indices(length):
    indices = list(range(length))
    np.random.shuffle(indices)
    split = int(np.floor(0.1 * length))
    test_indices = indices[:split]
    return test_indices

tumor_dataset = dataset.WeedDataset(DATASET_PATH)

# train_indices, valid_indices, test_indices = [8, 17, 22, 33, 39, 59 ,60, 61, 65], [80, 81, 85], [112, 39, 59 ,60, 61, 65]
test_indices = sampler_indices(len(tumor_dataset))
test_sampler = SubsetRandomSampler(test_indices)

test_loader = torch.utils.data.DataLoader(tumor_dataset, batch_size=1, sampler=test_sampler)

FILTER_LIST = [16,32,64,128,256]

model = ResUNet.ResUNet(FILTER_LIST).to(device)
path = 'outputs/ResUNet.pt'

classifier = classifier.WeedClassifier(model, device)

if str(device) == 'cpu':
    classifier.model.load_state_dict(torch.load(path, map_location='cpu'))
else:
    classifier.model.load_state_dict(torch.load(path))

cpu


In [2]:
def pil2cv(pil):
    image = np.array(pil)
    image = image[:, :, ::-1].copy()
    return image


In [3]:
i=0

In [14]:
image_index = test_indices[i]
sample = tumor_dataset[image_index]
rgb, mask, output, score = classifier.predict(sample)

print('Score', score)

mask = Image.fromarray((mask * 255).astype(np.uint8))
output = Image.fromarray((output * 255).astype(np.uint8))

rgb = rgb.numpy().transpose(1, 2, 0)
mask = pil2cv(mask)
output = pil2cv(output)

cv2.imshow('RGB', rgb)
cv2.imshow('Original mask', mask)
cv2.imshow('Predicted mask', output)
cv2.waitKey(0)
cv2.destroyAllWindows()


i += 1
if i >= len(test_indices):
    i = 0 

Score 0.54998104227724
